In [2]:
import pandas as pd
from geopy import distance as distance
from geopy import Point as Point
from scipy.spatial.distance import cdist
import numpy as np
import scipy as stats
import datetime


In [3]:
taxi_trip=pd.read_csv('/content/nyc_taxi_trip_duration.csv')

FileNotFoundError: ignored

In [ ]:
taxi_trip.tail(10)

In [ ]:
taxi_trip['pickup_datetime']=pd.to_datetime(taxi_trip['pickup_datetime'])

In [ ]:
taxi_trip['time']=taxi_trip['pickup_datetime'].dt.time

In [ ]:
#taxi_trip.pickup_day=pd.to_datetime(taxi_trip.pickup_day)
taxi_trip['pickup_day']=taxi_trip['pickup_datetime'].apply(lambda x : x.day_name())
#taxi_trip['pickup_day']=taxi_trip['pickup_day'].dt.strftime('%a')

In [ ]:
taxi_trip['pickup_month']=taxi_trip['pickup_datetime'].apply(lambda x:x.month_name())
#taxi_trip.pickup_month=pd.to_datetime(taxi_trip.pickup_month)

In [ ]:
taxi_trip=taxi_trip.drop('time',axis=1)


In [ ]:
taxi_trip.head()

In [ ]:
taxi_trip['date_pickup']=taxi_trip['pickup_datetime'].dt.date

In [ ]:
taxi_trip['date_pickup']=pd.to_datetime(taxi_trip['date_pickup'])


In [ ]:
taxi_trip['month_pickup']=taxi_trip['date_pickup'].dt.month

In [ ]:
taxi_trip['time']=taxi_trip['pickup_datetime'].dt.time

In [ ]:
taxi_trip['hours']=taxi_trip['pickup_datetime'].dt.hour



In [ ]:
taxi_trip[]

In [ ]:
taxi_trip.head()

In [ ]:
rows,columns= taxi_trip.shape
rows,columns

Data cleaning before handling EDA

In [ ]:
taxi_trip.isnull().sum()

In [ ]:
taxi_trip.dropna(axis=0,how='any',inplace=True)

In [ ]:
taxi_trip.isnull().sum()

In [ ]:
rows,columns=taxi_trip.shape
rows,columns

In [ ]:
#Since these are locations and cannot be replaced we might have to drop the nulls , but for
# for the sake of data intergrity we have to ensure that the average remains the same after
#removing data.

taxi_trip.mean()

A Program that enables a user to calculate their distance........

In [ ]:



def calculate_distance() :

      pickup_lat=float(input('Please enter your pickup latitude '))
      pickup_long=float(input('Please enter your pickup longitude '))
      pickup_time=(pickup_lat,pickup_long)
      dropoff_lat=float(input('Please enter your dropoff latitude '))
      dropoff_long=float(input('Please enter your dropoff longitude '))
      drop_time=(dropoff_lat,dropoff_long)



      print('Your distance in kilometers from your pickup location to your dropoff location is  ',distance.distance(pickup_time,drop_time).kilometers ,'km' )


Calculating Distance

In [ ]:
dis=pd.DataFrame(taxi_trip['pickup_latitude'])

In [ ]:
dis=dis.join(taxi_trip['pickup_longitude'])

In [ ]:
dis=dis.join(taxi_trip['dropoff_latitude'])

In [ ]:
dis=dis.join(taxi_trip['dropoff_longitude'])

A merging of these attributes will assist us in finding the distance between our pickup location  and dropoff location.

In [ ]:
dis


In [ ]:
#Use Eucledian Geometry for Distance (sqrt(x2-x1)+(y2-y1))
import math
#rad=math.pi


dis['pickup_latitude']=((dis['pickup_latitude'])/180)*math.pi
dis['pickup_longitude']=((dis['pickup_longitude'])/180)*math.pi
dis['dropoff_latitude']=((dis['dropoff_latitude'])/180)*math.pi
dis['dropoff_longitude']=((dis['dropoff_longitude'])/180)*math.pi




In [ ]:
dis['pickup_latitude'].head(2)


In [ ]:
dis['pickup_latitude']

In [ ]:
dis['pickup_latitude']

In [ ]:
dis['latitude**2']=(dis['dropoff_latitude']-dis['pickup_latitude'])**2
dis['latitude**2']

In [ ]:
dis['longitude**2']=(dis['dropoff_longitude']-dis['pickup_longitude'])**2

In [ ]:
import math as math

dis['distance_added']=dis['latitude**2']+dis['longitude**2']

In [ ]:


# Trying to square root the distance _added so we can find the distance



distance_in_rads=np.sqrt(dis['distance_added'])

#dis_tance

In [ ]:
distance_in_rads=pd.DataFrame(distance_in_rads)

In [ ]:
distance_in_rads=distance_in_rads*6371.0

# Converting into radians

Since we have the distance in rads we need to change it into km::

distance_in_rads

Since we have the distance in rads we need to change it into km::

We change it by finding its product with the earths radius

In [ ]:
distance_in_rads

In [ ]:
distance_in_rads['time_now']=pd.to_timedelta(taxi_trip['trip_duration'],unit='s')


In [ ]:
distance_in_rads

In [ ]:
hi=pd.DataFrame(taxi_trip['store_and_fwd_flag'].value_counts())

The graph below shows that there is a linear relationship between the distance travelled as well as time , we need to continue with our analysis to check on the stregnth of the relationship. Our graph also shows us that there are some data points that are moving in a certain directin which shows linear relationship between the two variables

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('whitegrid')
#axes = plt.axes(projection='3d')
plt.figure(figsize=(20,8))
plt.title('The relationship between the distance travelled and time in seconds')
sns.scatterplot(x=distance_in_rads['distance_added'],y=taxi_trip['trip_duration'],color='purple')


The Graph also shows us that the distance attribute is positively skewed as we can see from our kde plot

In [ ]:
plt.figure(figsize=(20,8))
sns.kdeplot(distance_in_rads['distance_added'],color='purple')



We will use correlation to check the stregnth of the relationship between the two variables  which is distance and time

In [ ]:
correlation = taxi_trip['trip_duration'].corr(distance_in_rads['distance_added'])

print("The correlation coefficient is  : " , correlation )

In conclusion for distance and time have a weak positive relationship

Box and whisker for distance

Our box and whisker shows that we have an amount of 4-10 passengers that are far from our average pasengers

In [ ]:
sns.set_palette("cubehelix")
sns.boxplot(distance_in_rads['distance_added'],orient='verticle',color='purple')
plt.xlabel('Box and Whisker for distance travelled')

My Hypothesis:


1.   Duration may take a bit longer during  
     weekends

2.   The distance affects the duration of the
     trip
3.  Trip duration is affected by time     






Bivariate analysis

In [ ]:
sns.set_palette("cubehelix")
sns.barplot(x=taxi_trip['vendor_id'].value_counts(),y=taxi_trip['trip_duration'],color='purple')
plt.title('Vendors vs Trip Duration')


There is a large number of flags that are submitted to the server which
results in a lesser amount of distance covered per vendor due to the lack of communication between the customer and the vendor  .

(This represents an indirect relationship between the two)


When there is a small amount of store and forward flags submitted to a server it results in a smaller of distance covered by the vendor.

(This represents a direct relationship between the two )




In [ ]:
#custom_palette = ["#3498db", "#e74c3c", "#2ecc71", "#f1c40f", "#9b59b6"]



sns.set_palette(palette='Set5',n_colors=10)
#sns.set_palette(sns.color_palette(custom_palette))
#plt.figure(figsize=(20,8))
sns.displot(x=taxi_trip['store_and_fwd_flag'],y=distance_in_rads['distance_added'],color='purple')
#plt.show()

According to our graph we can see that Sunday is the most travelled day according to our bar graph

In [ ]:
sns.set_palette("cubehelix")
plt.xticks(rotation=40)



sns.barplot(x=taxi_trip['pickup_day'],y=distance_in_rads['distance_added'],hue=taxi_trip['pickup_day'])



According to the graph month that most people travelled in is May and people travel less during the month of June

In [ ]:
#trip per month
sns.set_palette('cubehelix')
plt.xticks(rotation=40)
sns.barplot(x=taxi_trip['pickup_month'],y=distance_in_rads['distance_added'])
plt.title('A bar graph for trip taken per month')

Analysis:



In [ ]:
#Duration per month
sns.set_palette('cubehelix')
sns.set_context('paper')
sns.set_theme('notebook')

sns.barplot(y=taxi_trip['trip_duration'],x=taxi_trip['vendor_id'])
plt.title('A Graph showing trip duration per vendor ID')

Analysis ⤵
The longest distance was covered during a passenger count of 6   


In [ ]:
sns.set_palette('rainbow')
plt.title('Distance covered per passenger')
sns.set_context('paper')

sns.set_theme('notebook')
sns.barplot(x=taxi_trip['passenger_count'],y=distance_in_rads['distance_added'],hue=taxi_trip['passenger_count'])

In [ ]:
plt.title('A graph for Duration vs distance ')
sns.set_palette(palette='Set1', n_colors=2)
sns.scatterplot(x=distance_in_rads['distance_added'],y=taxi_trip['trip_duration'],data=taxi_trip,markers='o')


In [ ]:
sns.boxenplot(taxi_trip['trip_duration'],color='blue')
plt.title('Box and Whisker plot for Trip duration ')

In [ ]:


sns.barplot(x=taxi_trip['hours'],y=distance_in_rads['distance_added'])
sns.set_palette('cubehelix')
plt.title('Distance travelled per Hour')

In this case according to my hypothesis time duration is affected by the following :

1.   Day _

2.   Distance

3.   Time


**According to the graph the day that people travel in the most is on Sunday, which has proved the hypothesis to be true that most people travel on weekends hence it affects the trip duration.**


**The time that people travel the most is at fifth hour , which means if you travel at that time your trip duration will be affected .**


**We noticed from the scatterplot that distance has a linear relationship with the trip duration with a weak positive relationship**







In [ ]:
#from geopy.format import DISTANCE_FORMAT


n = len(distance_in_rads['distance_added'])
mean_x = sum(distance_in_rads['distance_added']) / n
mean_y = sum(taxi_trip['trip_duration']) / n
slope = sum((distance_in_rads['distance_added'][i] -distance_in_rads['distance_added'].mean() ) * (taxi_trip['trip_duration'][i] - taxi_trip['trip_duration'].mean()) for i in range(n)) / sum((distance_in_rads['distance_added'][i] - distance_in_rads['distance_added'].mean()) ** 2 for i in range(n))



In [ ]:
print("The gradient between duration and distance is", slope)

In conclusion when the distance is increased in kilometers ,we can expect the the time duration to increase by 116.38500061076375 in seconds .

Hence time duration is affected by the distance.